In [ ]:
import cv2
import numpy as np
import scipy
from scipy import signal
import scipy.ndimage as nd
import matplotlib.pyplot as plt

# Load and preprocess image
img = cv2.imread(r"/content/download (5).jpeg")
original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gaussian = cv2.GaussianBlur(gray, (3, 3), 0)

# Edge detection - Canny
img_canny = cv2.Canny(img, 100, 200)

# Edge detection - Sobel
img_sobelx = cv2.Sobel(img_gaussian, cv2.CV_8U, 1, 0, ksize=5)
img_sobely = cv2.Sobel(img_gaussian, cv2.CV_8U, 0, 1, ksize=5)
img_sobel = img_sobelx + img_sobely

# Edge detection - Prewitt
kernelx = np.array([[1,1,1], [0,0,0], [-1,-1,-1]])
kernely = np.array([[-1,0,1], [-1,0,1], [-1,0,1]])
img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)
img_prewitt = img_prewittx + img_prewitty

# Laplacian of Gaussian
img1 = cv2.GaussianBlur(gray, (3, 3), 0)
laplacian = cv2.Laplacian(img1, cv2.CV_64F)

img2 = cv2.imread(r"/content/download (5).jpeg", 0)
LoG = nd.gaussian_laplace(img2, 2)
thres = np.absolute(LoG).mean() * 0.75

# Use numpy.zeros instead of scipy.zeros
output = np.zeros(LoG.shape)
w = output.shape[1]
h = output.shape[0]

# Zero crossing detection
for y in range(1, h - 1):
    for x in range(1, w - 1):
        patch = LoG[y-1:y+2, x-1:x+2]
        p = LoG[y, x]
        maxP = patch.max()
        minP = patch.min()
        if (p > 0):
            zeroCross = minP < 0
        else:
            zeroCross = maxP > 0
        if ((maxP - minP) > thres) and zeroCross:
            output[y, x] = 1

# Convolution with custom kernel
A = np.array([
    [0, 0, -1, -1, -1, 0, 0],
    [0, -2, -3, -3, -3, -2, 0],
    [-1, -3, 5, 5, 5, -3, -1],
    [-1, -3, 5, 16, 5, -3, -1],
    [-1, -3, 5, 5, 5, -3, -1],
    [0, -2, -3, -3, -3, -2, 0],
    [0, 0, -1, -1, -1, 0, 0]
], dtype=np.float32)

ratio = img2.shape[0] / 500.0
new_width = int(img2.shape[1] / ratio)
original = img2.copy()
nimg = cv2.resize(img2, (new_width, 500))

I1 = signal.convolve2d(nimg, A, mode='same')
I1 = np.absolute(I1)
I1 = (I1 - np.min(I1)) / float(np.max(I1) - np.min(I1))

# Apply Gaussian Blur for smoothing
I2 = cv2.GaussianBlur(I1, (5, 5), 0)

# Display all results
plt.subplot(2, 4, 1), plt.imshow(original_img)
plt.title("Original"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 2), plt.imshow(gray, cmap='gray')
plt.title("Grayscale"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 3), plt.imshow(img_canny, cmap='gray')
plt.title("Canny Edge"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 4), plt.imshow(img_sobel, cmap='gray')
plt.title("Sobel Edge"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 5), plt.imshow(img_prewitt, cmap='gray')
plt.title("Prewitt Edge"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 6), plt.imshow(laplacian, cmap='gray')
plt.title("Laplacian"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 7), plt.imshow(output, cmap='gray')
plt.title("LoG Zero Crossing"), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 8), plt.imshow(I2, cmap='gray')
plt.title("Custom Filter + Blur"), plt.xticks([]), plt.yticks([])

plt.tight_layout()
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: bad operand type for abs(): 'NoneType'